In [1]:
import pickle
with open('q_table_3_8_file', 'rb') as handle:
    temp_q_table = pickle.load(handle)

In [1]:
from __future__ import division
import numpy as np

import malmo.MalmoPython as MalmoPython
import os
import random
import sys
import time
import json
import random
import math
import errno
from collections import defaultdict, deque
from timeit import default_timer as timer
import actions
import LimitedMap
import random_map
import LimitedMap2

missionXML= LimitedMap.missionXML1 + LimitedMap.limited_sphere() + LimitedMap.missionXML2
missionXML2 = random_map.missionXML1 + random_map.random_sphere() + random_map.missionXML2
missionXML3 = LimitedMap2.missionXML1 + LimitedMap2.limited_sphere() + LimitedMap2.missionXML2
blocks = ['bedrock', 'air', 'glass', 'purpur_block', 'gold_block', 'iron_block']


class agent(object):
    def __init__(self, alpha=0.3, gamma=1, n=1, q_table={}):
        """Constructing an RL agent.

        Args
            alpha:  <float>  learning rate      (default = 0.3)
            gamma:  <float>  value decay rate   (default = 1)
            n:      <int>    number of back steps to update (default = 1)
        """
        self.epsilon = 0.2  # chance of taking a random action instead of the best
        self.q_table = q_table
        self.n, self.alpha, self.gamma = n, alpha, gamma
        self.inventory = defaultdict(lambda: 0, {})



    @staticmethod
    def is_solution(agent_host):
        world_state = agent_host.getWorldState()
        msg = world_state.observations[-1].text
        
        observations = json.loads(msg)
        grid = observations.get(u'floor3x3x4', 0)
        idx = [4,10,12,14,16,19,21,23,25,31]
        if grid[31] == 'air':
            idx += [28,30,32,34]
            if grid[34] == 'air':
                idx += [33,35]
            if grid[32] == 'air':
                idx += [29,35]
            if grid[30] == 'air':
                idx += [33,27]
            if grid[28] == 'air':
                idx += [27,29]
        
        if grid[25] == 'air':
            idx += [24,26,34]
            if grid[26] == 'air':
                idx += [35]
            if grid[24] == 'air':
                idx += [33]
        if grid[23] == 'air':
            idx += [20,26,32]
            if grid[26] == 'air':
                idx += [35]
            if grid[20] == 'air':
                idx += [29]
        if grid[21] == 'air':
            idx += [18,24,30]
            if grid[24] == 'air':
                idx += [33]
            if grid[18] == 'air':
                idx += [27]
        if grid[19] == 'air':
            idx += [18,20,28]
            if grid[20] == 'air':
                idx += [29]
            if grid[18] == 'air':
                idx += [27]
        if grid[16] == 'air':
            idx += [15,17,25]
            if grid[17] == 'air':
                idx += [8]
            if grid[15] == 'air':
                idx += [6]
        if grid[14] == 'air':
            idx += [11,17,23]
            if grid[11] == 'air':
                idx += [2]
            if grid[17] == 'air':
                idx += [8]
        if grid[12] == 'air':
            idx += [9,15,21]
            if grid[9] == 'air':
                idx += [0]
            if grid[15] == 'air':
                idx += [6]
        if grid[10] == 'air':
            idx += [9,11,19]
            if grid[9] == 'air':
                idx += [0]
            if grid[11] == 'air':
                idx += [2]
        
        for i in idx:
            if grid[i] == 'diamond_block':
                return True
        return False


    def get_curr_state(self, grid):
        """Creates a unique identifier for a state.

        The state is defined as the items in the agent inventory. Notice that the state has to be sorted -- otherwise
        differnt order in the inventory will be different states.
        """
#         world_state = agent_host.getWorldState()

#         msg = world_state.observations[-1].text
#         observations = json.loads(msg)
#         grid = observations.get(u'floor3x3x4', 0)
        grid = tuple(grid)
        return tuple((grid, ))
    
    def get_curr_state_2(self, grid):
        """Creates a unique identifier for a state.

        The state is defined as the items in the agent inventory. Notice that the state has to be sorted -- otherwise
        differnt order in the inventory will be different states.
        """
#         world_state = agent_host.getWorldState()

#         msg = world_state.observations[-1].text
#         observations = json.loads(msg)
#         grid = observations.get(u'floor3x3x4', 0)
        #grid = tuple(grid)
        
        north = [grid[6],grid[7],grid[8],grid[15],grid[16],grid[17],grid[24],grid[25],grid[26],grid[33],grid[34],grid[35]]
        south = [grid[0],grid[1],grid[2],grid[9],grid[10],grid[11],grid[18],grid[19],grid[20],grid[27],grid[28],grid[29]]
        west = [grid[2],grid[5],grid[8],grid[11],grid[14],grid[17],grid[20],grid[23],grid[26],grid[29],grid[32],grid[35]]
        east = [grid[0],grid[3],grid[6],grid[9],grid[12],grid[15],grid[18],grid[21],grid[24],grid[27],grid[30],grid[33]]
        down = [grid[0],grid[1],grid[2],grid[3],grid[4],grid[5],grid[6],grid[7],grid[8]]
        up = [grid[27],grid[28],grid[29],grid[30],grid[31],grid[32],grid[33],grid[34],grid[35]]
        
        # bedrock air glass purpur_block gold_block iron_block diamond_block
        n = [0,0,0,0,0,0]
        s = [0,0,0,0,0,0]
        w = [0,0,0,0,0,0]
        e = [0,0,0,0,0,0]
        u = [0,0,0,0,0,0]
        d = [0,0,0,0,0,0]
        for i in north:
            for j in range(len(blocks)):
                if i == blocks[j]:
                    n[j] += 1
        
        for i in south:
            for j in range(len(blocks)):
                if i == blocks[j]:
                    s[j] += 1
        
        for i in west:
            for j in range(len(blocks)):
                if i == blocks[j]:
                    w[j] += 1
        
        for i in east:
            for j in range(len(blocks)):
                if i == blocks[j]:
                    e[j] += 1
                    
        for i in up:
            for j in range(len(blocks)):
                if i == blocks[j]:
                    u[j] += 1
                    
        for i in down:
            for j in range(len(blocks)):
                if i == blocks[j]:
                    d[j] += 1
        
        
        state = tuple([tuple(n),tuple(s), tuple(w), tuple(e), tuple(u), tuple(d)])
        return tuple((state, ))


    def choose_action(self, curr_state, possible_actions, eps):
        """Chooses an action according to eps-greedy policy. """
        if curr_state not in self.q_table:
            self.q_table[curr_state] = {}
        for action in possible_actions:
            if action not in self.q_table[curr_state]:
                self.q_table[curr_state][action] = 0
        
        if random.random() <= eps:
            #print('RANDOM!!!!!')
            a = possible_actions[random.randint(0, len(possible_actions) - 1)]
        else:
            temp = max(self.q_table[curr_state].values())
            list1 = []
            if temp > 0:
                print('best:',temp, 'all', self.q_table[curr_state].values())
            for k, v in self.q_table[curr_state].items():
                if v == temp:
                    list1.append(k)
            a = list1[random.randint(0, len(list1) - 1)]
        return a


    def act(self, agent_host, action, grid):
        #print(action + ",", end = " ")
        action(agent_host, grid)
        if self.is_solution(agent_host):
            return 10000
        world_state = agent_host.getWorldState()
        msg = world_state.observations[-1].text
        
        observations = json.loads(msg)
        grid = observations.get(u'floor3x3x4', 0)
        reward = 0
        for i in grid:
            if i == 'purpur_block':
                reward += 100
            elif i == 'gold_block':
                reward += 200
            elif i == 'iron_block':
                reward += 300
        return 0

    def update_q_table(self, tau, S, A, R, T):
        """Performs relevant updates for state tau.

        Args
            tau: <int>  state index to update
            S:   <dequqe>   states queue
            A:   <dequqe>   actions queue
            R:   <dequqe>   rewards queue
            T:   <int>      terminating state index
        """
        #curr_s, curr_a, curr_r = S.popleft(), A.popleft(), R.popleft()
        curr_s = S.popleft()
        curr_a = A.popleft()
        curr_r = R.popleft()
        G = sum([self.gamma ** i * R[i] for i in range(len(S))])
        if tau + self.n < T:
            G += self.gamma ** self.n * self.q_table[S[-1]][A[-1]]

        old_q = self.q_table[curr_s][curr_a]
        self.q_table[curr_s][curr_a] = old_q + self.alpha * (G - old_q)

    def best_policy(self, agent_host):
        """Reconstructs the best action list according to the greedy policy. """
        count = 0

        next_a = ""
        while True:
            count += 1
            time.sleep(0.1)
            world_state = agent_host.getWorldState()
            msg = world_state.observations[-1].text
            observations = json.loads(msg)
            grid = observations.get(u'floor3x3x4', 0)
            
            curr_state = self.get_curr_state(grid)
            possible_actions = actions.get_possible_actions(grid)
            next_a = self.choose_action(curr_state, possible_actions, 0.3)

            next_a(agent_host, grid)
            time.sleep(0.1)
            find = self.is_solution(agent_host)
            if find:
                print('Found at ', {count})
                break

        #print 'Best policy so far is %s with reward %.1f' % (policy, current_r)

    def run(self, agent_host):
        """Learns the process to compile the best gift for dad. """
        S, A, R = deque(), deque(), deque()
        present_reward = 0
        done_update = False
        while not done_update:
            # !!!!!!!!!!!!!!!!!
            time.sleep(0.1)
            world_state = agent_host.getWorldState()
            msg = world_state.observations[-1].text
            observations = json.loads(msg)
            grid = observations.get(u'floor3x3x4', 0)
            
            s0 = self.get_curr_state_2(grid)
            #print(s0)

            possible_actions = actions.get_possible_actions(grid)
            a0 = self.choose_action(s0, possible_actions, self.epsilon)
            S.append(s0)
            A.append(a0)
            R.append(0)

            T = 100
            p_score = 0
            for t in range(300):
                
                if (t%10) == 0:
                    #print(t)
                    pass
                #time.sleep(0.1)
                if t < T:
                    world_state = agent_host.getWorldState()
                    #current_r = self.act(agent_host, A[-1], S[-1])
                    
                    A[-1](agent_host, grid)
                    time.sleep(0.1)
                    find = self.is_solution(agent_host)
                    if find:
                        current_r = 10000
                    else:
                        time.sleep(0.1)
                        world_state = agent_host.getWorldState()
                        msg = world_state.observations[-1].text

                        observations = json.loads(msg)
                        grid = observations.get(u'floor3x3x4', 0)
                        score = 0
                        for i in grid:
                            if i == 'purpur_block':
                                score += 100
                            elif i == 'gold_block':
                                score += 200
                            elif i == 'iron_block':
                                score += 300
                            elif i == 'glass':
                                score += 1
                        if score > p_score:
                            reward = 1000
                        elif score < p_score:
                            reward = -1000
                        else:
                            reward = 0
                        p_score = score
                        current_r = reward
                    
                    R.append(current_r)

                    world_state = agent_host.getWorldState()

                    if find:
                        # Terminating state
                        print('found at ', t)
                        T = t + 1
                        S.append('Term State')
                        score = current_r
                        #print("Found! The score of this run is:", score)
                    else:
                        time.sleep(0.1)
                        # !!!!!!!!!!!!!!!!!
                        world_state = agent_host.getWorldState()
                        msg = world_state.observations[-1].text
                        observations = json.loads(msg)
                        grid = observations.get(u'floor3x3x4', 0)
                        
                        s = self.get_curr_state_2(grid)
                        #print(s)
                        S.append(s)
                        possible_actions = actions.get_possible_actions(grid)
                        next_a = self.choose_action(s, possible_actions, self.epsilon)
                        A.append(next_a)

                tau = t - self.n + 1
                if tau >= 0:
                    #print(t)
                    self.update_q_table(tau, S, A, R, T)

                if tau == T - 1:
                    #print('asdfadsf')
                    while len(S) > 1:
                        tau = tau + 1
                        self.update_q_table(tau, S, A, R, T)
                    done_update = True
                    break

sphere_count: 2 
 record_check: ['x,y,z: 6,15,-5 sizes: 9,7,2', 'x,y,z: 9,26,-8 sizes: 10,5,2']
sphere_count: 6 
 record_check: ['x,y,z: -5,29,-1 sizes: 10,4,2', 'x,y,z: 13,23,13 sizes: 6,5,2', 'x,y,z: -3,13,-9 sizes: 8,7,2', 'x,y,z: -8,26,0 sizes: 11,8,2', 'x,y,z: -1,23,1 sizes: 9,4,2', 'x,y,z: 10,19,1 sizes: 6,5,2']


In [ ]:
if __name__ == '__main__':
    random.seed(0)
    #sys.stdout = os.fdopen(sys.stdout.fileno(), 'w', 0)  # flush print output immediately
    print('Starting...', flush=True)

    expected_reward = 3390
    my_client_pool = MalmoPython.ClientPool()
    my_client_pool.add(MalmoPython.ClientInfo("127.0.0.1", 10000))

    

    num_reps = 30000
    n=1
#     agent1 = agent(n=n, q_table=temp_q_table)
    agent1 = agent(n=n)
    print("n=",n)

    for iRepeat in range(num_reps):
        agent_host = MalmoPython.AgentHost()
        try:
            agent_host.parse(sys.argv)
        except RuntimeError as e:
            #print('!!!!!!!!')
            #print('ERROR:', e)
            #print(agent_host.getUsage())
            exit(1)
        if agent_host.receivedArgument("help"):
            #print(agent_host.getUsage())
            exit(0)
        my_mission = MalmoPython.MissionSpec(missionXML3, True)
        my_mission_record = MalmoPython.MissionRecordSpec()  # Records nothing by default
        my_mission.requestVideo(800, 500)
        my_mission.setViewpoint(0)
        max_retries = 3
        for retry in range(max_retries):
            try:
                # Attempt to start the mission:
                agent_host.startMission(my_mission, my_client_pool, my_mission_record, 0, "Odie")
                break
            except RuntimeError as e:
                if retry == max_retries - 1:
                    print("Error starting mission", e)
                    print("Is the game running?")
                    exit(1)
                else:
                    time.sleep(2)

        world_state = agent_host.getWorldState()
        while not world_state.has_mission_begun:
            time.sleep(0.1)
            world_state = agent_host.getWorldState()

    
        print((iRepeat+1), 'Learning Q-Table:', end = " ")
        time.sleep(0.1)
        agent1.run(agent_host)
        #agent1.best_policy(agent_host)
        #agent_host.sendCommand('quit')

        time.sleep(1)

Starting...
n= 1
1 Learning Q-Table: best: 300.0 all dict_values([0, 0, 0, 300.0, 0])
best: 300.0 all dict_values([90.0, 0, 0, 300.0, 0])
best: 300.0 all dict_values([90.0, 0, 0, 300.0, 0])
best: 300.0 all dict_values([90.0, 0, 0, 300.0, 0])
best: 300.0 all dict_values([90.0, 0, 300.0, 210.0, 0])
best: 300.0 all dict_values([0, 0, 0, 0, 300.0, 0, 0, 0, 0])
best: 300.0 all dict_values([0, 0, 0, 0, 300.0])
best: 300.0 all dict_values([300.0, 0, 0, 0, 0])
best: 600.0 all dict_values([90.0, 0, 600.0, 210.0, 0])
best: 300.0 all dict_values([0, 0, 0, 300.0, 0, 0, 0, 0])
best: 300.0 all dict_values([0, 0, 300.0, 0, 0])
2 Learning Q-Table: best: 720.0 all dict_values([90.0, 0, 720.0, 210.0, 0])
best: 510.0 all dict_values([0, 0, 0, 0, 510.0, -300.0, 0, 0, 0])
best: 300.0 all dict_values([0, 0, 300.0, 0, 0])
best: 300.0 all dict_values([0, 0, 0, 300.0, 0, 0, -300.0, 0])
best: 300.0 all dict_values([300.0, 0, 0, 0, 0, 0, 0.0, 0])
best: 300.0 all dict_values([0.0, -300.0, -210.0, 0, 300.0, 0, 0, 

best: 831.93 all dict_values([-164.1, 0, 0, 0, 831.93, -300.0, -312.9, 0, -300.0])
best: 300.0 all dict_values([0, 0, 0, 0, 0, 300.0, 0, 0])
best: 300.0 all dict_values([0, 0, 0, 0, 300.0, 0, 0, 0])
best: 300.0 all dict_values([0, 0, 0, 0, 0, 300.0, -300.0, 0])
best: 1234.1496 all dict_values([0, 1234.1496, 0, 0, 0, 0, 0, 0])
best: 882.351 all dict_values([0, 0, 0, 882.351, 0, 0, 0, 0])
best: 300.0 all dict_values([300.0, 0, 0, 0, 0, 0, 0])
best: 453.0 all dict_values([0, 453.0, 0, 0, 0])
best: 300.0 all dict_values([0, 0, 0, 300.0, 0, 0, 0])
best: 917.6457 all dict_values([0, 0, 0, 917.6457, 0, 0, 0, 0])
best: 210.0 all dict_values([0, 210.0, 0, 0, -300.0, 0, -107.14799999999998, 0])
12 Learning Q-Table: best: 1964.0726253926102 all dict_values([90.0, 0, 1964.0726253926102, 210.0, 516.0])
best: 1450.38952257 all dict_values([0, 0, -147.0, 0, 1450.38952257, -510.0, 0, 0, -210.0])
best: 882.351 all dict_values([-164.1, 0, 0, 0, 882.351, -300.0, -312.9, 0, -300.0])
best: 300.0 all dict_v

best: 1259.058 all dict_values([0, 0, -300.0, 0, 1259.058, 0, 0, -300.0, -210.0])
16 Learning Q-Table: best: 2508.6316435609706 all dict_values([90.0, 0, 2508.6316435609706, 210.0, 516.0])
best: 1969.22835970834 all dict_values([0, 0, -147.0, 0, 1969.22835970834, -510.0, 0, 0, -210.0])
best: 1235.8563425700002 all dict_values([-164.1, 0, 0, 0, 1235.8563425700002, -300.0, -312.9, 0, -300.0])
best: 848.9067 all dict_values([-300.0, -300.0, -210.0, -300.0, 848.9067, -510.0, -435.03, -300.0, -300.0])
best: 848.9067 all dict_values([-300.0, -300.0, -210.0, -300.0, 848.9067, -510.0, -435.03, -300.0, -300.0])
best: 848.9067 all dict_values([-300.0, -300.0, -210.0, -300.0, 848.9067, -510.0, -435.03, -300.0, -300.0])
best: 657.0 all dict_values([0, 657.0, 0, -300.0, 0, 0, -147.0, 0, 0])
best: 300.0 all dict_values([0, 0, 0, 0, 0, 300.0, 0, 0])
best: 156.86999999999998 all dict_values([0, 0, -300.0, 0, 0, 156.86999999999998, 0, 0])
best: 666.03 all dict_values([0, 0, 0, 0, 0, 666.03, 0, 0])
best

tuple([[1,2],2])

{(((0, 0, 7, 3, 2, 0),
   (0, 8, 2, 1, 1, 0),
   (0, 3, 7, 2, 0, 0),
   (0, 3, 4, 2, 3, 0),
   (0, 2, 7, 0, 0, 0),
   (0, 0, 1, 3, 5, 0)),): {<function actions.jump_east(agent, obs_grid)>: 0,
  <function actions.jump_south(agent, obs_grid)>: 0,
  <function actions.jump_west(agent, obs_grid)>: 0,
  <function actions.move_down(agent, obs_grid)>: 0,
  <function actions.move_east(agent, obs_grid)>: 0,
  <function actions.move_north(agent, obs_grid)>: -510.0,
  <function actions.move_south(agent, obs_grid)>: 0,
  <function actions.move_west(agent, obs_grid)>: 0},
 (((0, 6, 2, 2, 2, 0),
   (0, 0, 9, 2, 1, 0),
   (0, 4, 2, 3, 3, 0),
   (0, 2, 8, 2, 0, 0),
   (0, 5, 4, 0, 0, 0),
   (0, 0, 1, 3, 5, 0)),): {<function actions.jump_east(agent, obs_grid)>: 0,
  <function actions.jump_north(agent, obs_grid)>: 0,
  <function actions.jump_west(agent, obs_grid)>: 0,
  <function actions.move_down(agent, obs_grid)>: 300.0,
  <function actions.move_east(agent, obs_grid)>: 0,
  <function actions.move_north

In [6]:
import pickle
  
try:
    file = open('q_table_3_8_file', 'wb')
    pickle.dump(agent1.q_table, file)
    file.close()
  
except:
    print("Something went wrong")

In [7]:
import pickle
with open('q_table_3_8_file', 'rb') as handle:
    temp_q_table = pickle.load(handle)

In [8]:
temp_q_table

{(((0, 0, 7, 3, 2, 0),
   (0, 8, 2, 1, 1, 0),
   (0, 3, 7, 2, 0, 0),
   (0, 3, 4, 2, 3, 0),
   (0, 2, 7, 0, 0, 0),
   (0, 0, 1, 3, 5, 0)),): {<function actions.jump_east(agent, obs_grid)>: 0,
  <function actions.jump_south(agent, obs_grid)>: 0,
  <function actions.jump_west(agent, obs_grid)>: 0,
  <function actions.move_down(agent, obs_grid)>: 0,
  <function actions.move_east(agent, obs_grid)>: 0,
  <function actions.move_north(agent, obs_grid)>: -510.0,
  <function actions.move_south(agent, obs_grid)>: 0,
  <function actions.move_west(agent, obs_grid)>: 0},
 (((12, 0, 0, 0, 0, 0),
   (0, 8, 4, 0, 0, 0),
   (4, 2, 6, 0, 0, 0),
   (4, 4, 4, 0, 0, 0),
   (3, 4, 2, 0, 0, 0),
   (3,
    2,
    4,
    0,
    0,
    0)),): {<function actions.jump_east(agent, obs_grid)>: -300.0,
  <function actions.jump_west(agent, obs_grid)>: 0,
  <function actions.move_down(agent, obs_grid)>: 0,
  <function actions.move_east(agent, obs_grid)>: 0,
  <function actions.move_north(agent, obs_grid)>: 0,
  <funct